In [1]:
# Cell 1: Cài đặt đồng bộ
!pip install -q -U torch torchvision torchaudio transformers peft datasets bitsandbytes trl accelerate pandas scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 102.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 20.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 29.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5

## Local Inference on GPU 
Model page: https://huggingface.co/Qwen/Qwen3-4B-Instruct-2507

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/Qwen/Qwen3-4B-Instruct-2507)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [2]:
import os
import gc
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from datasets import Dataset, DatasetDict

# Import Transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    BitsAndBytesConfig, 
    TrainingArguments
)

# Import PEFT & TRL
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from sklearn.metrics import accuracy_score, classification_report

try:
    del model
    del trainer
    del tokenizer
except:
    pass
gc.collect()
torch.cuda.empty_cache()

# Environment config
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

2025-12-15 04:09:15.118667: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765771755.318773      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765771755.375455      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [ ]:
# Load model
MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"
NEW_MODEL_NAME = "Qwen-ViMedNLI-Hallucination-Detection"

# Config data path
DATA_PATH = "/kaggle/input/processed/processed_data_3labels/ViMedNLI/csv_processed/"
TRAIN_FILE = os.path.join(DATA_PATH, "train.csv")
DEV_FILE = os.path.join(DATA_PATH, "dev.csv")
TEST_FILE = os.path.join(DATA_PATH, "test.csv")

# Thiết lập device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Kiểm tra xem import có thành công không
print("Transformers version:", torch.__version__)

Using device: cuda
Transformers version: 2.9.1+cu128


In [ ]:
# Load and process function
def load_and_process_data(train_path, dev_path, test_path):
    # Read CSV
    df_train = pd.read_csv(train_path)
    df_dev = pd.read_csv(dev_path)
    df_test = pd.read_csv(test_path)

    label_map = {
        0: "Entailment",
        1: "Extrinsic-Hal",
        2: "Intrinsic-Hal"
    }

    def format_instruction(row):
        prompt = f"""You are a medical AI assistant. Your task is to determine the logical relationship between the following Context and Statement.
        
        Context: {row['sentence1']}
        Statement: {row['sentence2']}

        Based on the context, classify the statement as one of the followings:
        - Entailment: The statement is definitely true based on the context.
        - Extrinsic-hal: The truth of the statement cannot be determined from the context (it might be true or false, but the context doesn’t say).
        - Intrinsic-hal: The statement is definitely false or contradicts the context.

        Answer:
        """
        return prompt

    
    def process_df(df):
        data = []
        for _, row in df.iterrows():
            user_content = format_instruction(row)
            assistant_content = label_map.get(row['label'], str(row['label']))

            messages = [
                {"role": "user", "content": user_content},
                {"role": "assistant", "content": assistant_content}
            ]

            data.append({"messages": messages, "label_str": assistant_content})
        return Dataset.from_list(data)

    train_dataset = process_df(df_train)
    dev_dataset = process_df(df_dev)
    test_dataset = process_df(df_test)

    return DatasetDict({"train": train_dataset, "dev": dev_dataset, "test": test_dataset})

# Load data
dataset = load_and_process_data(TRAIN_FILE, DEV_FILE, TEST_FILE)
print("Sample train data", dataset['train'][0])

Sample train data {'messages': [{'content': 'You are a medical AI assistant. Your task is to determine the logical relationship between the following Context and Statement.\n        \n        Context: các thử nghiệm đáng chú ý với cr giá tiền baseline giá tiền trên mỗi hồ sơ cũ vầ lactate giá tiền\n        Statement: bệnh nhân có nồng độ cr giá tiền mol l tăng cr\n\n        Based on the context, classify the statement as one of the followings:\n        - Entailment: The statement is definitely true based on the context.\n        - Neutral: The truth of the statement cannot be determined from the context.\n        - Contradiction: The statement is definitely false based on the context.\n\n        Answer:\n        ', 'role': 'user'}, {'content': 'entailment', 'role': 'assistant'}], 'label_str': 'entailment'}


In [5]:
# Config Quantization (T4 bắt buộc dùng float16)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16, 
    bnb_4bit_use_double_quant=False,
)

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load Model (QUAN TRỌNG: torch_dtype=torch.float16)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16  # <--- Ép kiểu ngay từ khi load
)

# Tắt cache và chuẩn bị model
model.config.use_cache = False 
model.config.pretraining_tp = 1
model = prepare_model_for_kbit_training(model)

# --- 2. Cấu hình LoRA ---
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=16, 
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/238 [00:00<?, ?B/s]

In [6]:
# --- 3. Training Arguments ---
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=100,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,   # T4 dùng FP16
    bf16=False,  # T4 KHÔNG hỗ trợ BF16 -> Phải là False
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none",
    gradient_checkpointing=True 
)

# --- 4. Khởi tạo Trainer ---
# (Giả sử biến 'dataset' đã được tạo từ cell load data trước đó)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    eval_dataset=dataset['dev'],
    peft_config=peft_config,
    args=training_args,
    processing_class=tokenizer,
)

Tokenizing train dataset:   0%|          | 0/11232 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/11232 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/1395 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1395 [00:00<?, ? examples/s]

In [ ]:
print("Applying Hybrid Cast for T4 compatibility...")

# Duyệt qua tất cả tham số
for name, param in trainer.model.named_parameters():
    # TRƯỜNG HỢP 1: Tham số cần train (LoRA) -> Bắt buộc Float32 để tránh lỗi 'unscale FP16'
    if param.requires_grad:
        if param.dtype != torch.float32:
            param.data = param.data.to(torch.float32)
            print(f"Upcast trainable param {name} to float32")
            
    # TRƯỜNG HỢP 2: Tham số đóng băng (Base Model) -> Bắt buộc Float16 để tránh lỗi T4 không hỗ trợ BFloat16
    elif param.dtype == torch.bfloat16:
        param.data = param.data.to(torch.float16)
        print(f"Downcast frozen param {name} to float16")

# Xử lý buffers (thường là mask hoặc position ids)
for name, buffer in trainer.model.named_buffers():
    if buffer.dtype == torch.bfloat16:
        buffer.data = buffer.data.to(torch.float16)

print("Hybrid Cast complete. LoRA is FP32, Base is FP16/INT4.")

# --- 6. Bắt đầu train ---
trainer.train()

# --- 7. Lưu model ---
trainer.model.save_pretrained(NEW_MODEL_NAME)
tokenizer.save_pretrained(NEW_MODEL_NAME)

Applying Hybrid Cast for T4 compatibility...
Upcast trainable param base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight to float32
Upcast trainable param base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight to float32
Upcast trainable param base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight to float32
Upcast trainable param base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight to float32
Upcast trainable param base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight to float32
Upcast trainable param base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight to float32
Upcast trainable param base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight to float32
Upcast trainable param base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight to float32
Upcast trainable param base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight to float32
Upcast traina

In [ ]:
# --- 1. Dọn dẹp bộ nhớ VRAM (Quan trọng) ---
try:
    del model
    del trainer
except:
    pass
gc.collect()
torch.cuda.empty_cache()

print("Đã giải phóng VRAM. Bắt đầu reload model...")

# --- 2. Cấu hình đường dẫn (SỬA LỖI TẠI ĐÂY) ---
# Định nghĩa lại tên model
MODEL_NAME = "Qwen/Qwen3-4B-Instruct-2507"
NEW_MODEL_NAME = "Qwen-ViMedNLI-Hallucination-Detection"

# QUAN TRỌNG: Thêm "./" để chỉ định đây là thư mục local, không phải HuggingFace Repo
ADAPTER_PATH = f"./{NEW_MODEL_NAME}"

# Kiểm tra xem folder có tồn tại không để debug
if os.path.exists(ADAPTER_PATH):
    print(f"✅ Đã tìm thấy adapter tại local: {ADAPTER_PATH}")
else:
    print(f"⚠️ CẢNH BÁO: Không tìm thấy thư mục '{ADAPTER_PATH}'. Hãy kiểm tra lại cell training đã chạy xong chưa.")

# --- 3. Load lại Base Model (4-bit) ---
# Phải khai báo lại bnb_config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16
)

# --- 4. Gắn Adapter (LoRA) từ Local ---
# Sử dụng ADAPTER_PATH thay vì NEW_MODEL_NAME
try:
    model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
    model.eval() # Chuyển sang chế độ đánh giá
    print("Load Adapter thành công!")
except Exception as e:
    print(f"Lỗi khi load adapter: {e}")

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# --- 5. Hàm dự đoán ---
def predict(messages, model, tokenizer):
    user_prompt = messages[0]['content'] 
    
    text = tokenizer.apply_chat_template(
        [{"role": "user", "content": user_prompt}],
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer([text], return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs, 
            max_new_tokens=10, 
            pad_token_id=tokenizer.eos_token_id
        )
        
    generated_ids = outputs[0][inputs.input_ids.shape[1]:]
    response = tokenizer.decode(generated_ids, skip_special_tokens=True)
    return response.strip()

# --- 6. Chạy Evaluation ---
y_true = []
y_pred = []

print("Starting evaluation on Test set...")
# Đảm bảo biến dataset vẫn còn tồn tại từ các cell trước
test_subset = dataset['test'] 

for i in tqdm(range(len(test_subset))):
    sample = test_subset[i]
    ground_truth = sample['label_str']
    messages = sample['messages']
    
    try:
        prediction = predict(messages, model, tokenizer)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        prediction = "Error"
    
    y_true.append(ground_truth)
    y_pred.append(prediction)

# --- 7. Đánh giá và lưu kết quả (Dạng Phần Trăm) ---
def clean_pred(pred):
    pred = str(pred).lower().strip()
    if "Entailment" in pred: return "Entailment"
    if "Intrinsic-Hal" in pred: return "Intrinsic-Hal"
    if "Extrinsic-Hal" in pred: return "Extrinsic-Hal"
    return "unknown"

y_pred_clean = [clean_pred(p) for p in y_pred]

# Định nghĩa các nhãn cần đánh giá
target_names = ["Entailment", "Intrinsic-Hal", "Extrinsic-Hal"]

# 1. Lấy báo cáo dưới dạng Dictionary
report_dict = classification_report(y_true, y_pred_clean, labels=target_names, output_dict=True)

# 2. Chuyển sang DataFrame để xử lý
df_report = pd.DataFrame(report_dict).transpose()

# 3. Nhân 100 cho các cột Precision, Recall, F1-Score để ra phần trăm
cols_to_scale = ['precision', 'recall', 'f1-score']
df_report[cols_to_scale] = df_report[cols_to_scale] * 100

# 4. Định dạng cột 'support' về dạng số nguyên (tránh bị hiển thị 474.000)
df_report['support'] = df_report['support'].astype(int)

# 5. In ra màn hình với định dạng 3 chữ số sau dấu phẩy
print("\nClassification Report (Unit: %):")
print("-" * 60)
# float_format="{:.3f}".format giúp làm tròn 3 chữ số
print(df_report.to_string(float_format="{:.3f}".format))
print("-" * 60)

# 6. Lưu kết quả chi tiết ra CSV
df_result = pd.DataFrame({
    "Ground_Truth": y_true, 
    "Prediction_Raw": y_pred, 
    "Prediction_Clean": y_pred_clean
})
df_result.to_csv("submission_results.csv", index=False)
print("Results saved to submission_results.csv")

# 7. (Tùy chọn) Lưu bảng điểm ra CSV riêng
df_report.to_csv("classification_report_percentage.csv")
print("Report saved to classification_report_percentage.csv")

Đã giải phóng VRAM. Bắt đầu reload model...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

ValueError: Can't find 'adapter_config.json' at 'Qwen-ViMedNLI-Hallucination-Detection'